In [ ]:
from shapely.geometry import mapping
import geopandas as gpd
import numpy as np
import tqdm

In [ ]:
casas=gpd.read_file(r"C:/Users/ASUS/salida/shape/Tlane_new_casas.shp")
casas=casas.reset_index()
casas=casas.rename(columns={"index":"id"})

In [ ]:
casas["centroid_x"]=casas["geometry"].centroid.x
casas["centroid_y"]=casas["geometry"].centroid.y
casas=casas.drop_duplicates(subset=["centroid_x","centroid_y"]).reset_index(drop=True)

In [ ]:
distancia_m=20
geometry_a=[]
i=3081
for i in tqdm.tqdm(range(len(casas))):
    casas["dis_x"]=np.abs(casas["centroid_x"]-casas["centroid_x"][i])
    casas["dis_y"]=np.abs(casas["centroid_y"]-casas["centroid_y"][i])
    lista=[str(x) for x in list(casas[(casas["dis_x"]<distancia_m)&(casas["dis_y"]<distancia_m)].index)]
    casas.loc[i,"cercanos"]=",".join(lista)
    casas.loc[i,"cantidad"]=len(lista)
    if len(lista)>1:
        for j in range(1,len(lista)):
            s=gpd.GeoSeries(casas.loc[int(lista[0]),"geometry"])
            s1=gpd.GeoSeries(casas.loc[int(lista[j]),"geometry"])
            intersec=s.intersection(s1)[0]
            if len(mapping(intersec).get("coordinates"))>0 and mapping(intersec).get("type")=="Polygon":
                geometry_a.append(intersec)



In [ ]:
a=gpd.GeoDataFrame(geometry=geometry_a,crs=3857)
a.to_file("inters.shp")